In [1]:
import datasets
import utils
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.grid_search import GridSearchCV
import pickle_utils as pu

import tensorflow as tf
import GPflow

/home/adria/.virtualenvs/tf3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/adria/.virtualenvs/tf3/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [21]:
def r_squared(y_true, y):
    u = np.sum((y-y_true)**2)
    v = np.sum((y_true - np.mean(y_true))**2)
    return 1 - u/v

In [4]:
all_dsets = datasets.datasets(exclude_labels=False)
all_dsets["BostonHousing"][0].info()

+++ Importing Ionosphere
V2 ; must have more than 1 possible value
+++ Importing BreastCancer
+++ Importing Servo
+++ Importing BostonHousing
+++ Importing Soybean
+++ Importing Shuttle
+++ Importing LetterRecognition
<class 'pandas.core.frame.DataFrame'>
Index: 506 entries, 1 to 506
Data columns (total 14 columns):
crim       506 non-null float64
zn         506 non-null float64
indus      506 non-null float64
chas       506 non-null int32
nox        506 non-null float64
rm         506 non-null float64
age        506 non-null float64
dis        506 non-null float64
rad        506 non-null float64
tax        506 non-null float64
ptratio    506 non-null float64
b          506 non-null float64
lstat      506 non-null float64
medv       506 non-null float64
dtypes: float64(13), int32(1)
memory usage: 57.3+ KB


In [5]:
dset = all_dsets["BostonHousing"][0]
test_mask = np.random.rand(len(dset)) < 0.2
y_train = dset.values[~test_mask, -1]
X_train = dset.values[~test_mask, :-1]

ym, ys = y_train.mean(), y_train.std()
Xm, Xs = X_train.mean(axis=0), X_train.std(axis=0)

y_train = (y_train-ym)/ys
X_train = (X_train-Xm)/Xs

y_test = (dset.values[test_mask, -1]-ym)/ys
X_test = (dset.values[test_mask, :-1]-Xm)/Xs

In [14]:
rf = RandomForestRegressor(n_estimators=100, n_jobs=-1,
                           max_features=int(np.floor(X_train.shape[1]**.5)),
                           bootstrap=False,
                           min_samples_split=5)
rf.fit(X_train, y_train)
train_perf = rf.score(X_train, y_train)
test_perf = rf.score(X_test, y_test)
print("Test: {:.4f}, Training: {:.4f}".format(test_perf, train_perf))

Test: 0.8412, Training: 0.9939


In [31]:
n_features = X_train.shape[1]
k = GPflow.kernels.Matern12(n_features, ARD=True)
m = GPflow.gpr.GPR(X_train, y_train[:, np.newaxis], kern=k)
m.likelihood.variance = 0.01

In [35]:
def logger(x):
    if (logger.i % 10) == 0:
        print(x, m._objective(x)[0])
    logger.i+=1
logger.i = 0

m.optimize(method=tf.train.GradientDescentOptimizer(0.02), callback=logger)


[  5.96620292  11.08556274   9.08417628  15.00692098   1.82370147
   5.70221474   7.48657968   5.84922756   5.06739502   2.68437966
   8.78792001   7.65111614   3.27425244  -0.15995746  -8.78678087] 156.983096425
[  5.98328666  11.16286605   9.15161415  15.12873654   1.82774141
   5.7253437    7.51550292   5.87726846   5.09233655   2.69387595
   8.8428824    7.70959931   3.28748412  -0.15581031  -8.80583104] 156.805525162
[  5.99990074  11.23912452   9.21821311  15.2490712    1.83172971
   5.74808083   7.54392396   5.90485825   5.11682359   2.70322492
   8.89704911   7.76748698   3.30050342  -0.15173376  -8.82451237] 156.632553938
[  6.01606491  11.31437196   9.28399969  15.36797104   1.83566722
   5.77043981   7.57186199   5.93201236   5.14087238   2.71243105
   8.95044595   7.82479508   3.31331655  -0.1477106   -8.84283883] 156.463976131
[  6.03179666  11.38864043   9.34899896  15.48548022   1.83954533
   5.79242732   7.59933253   5.95874287   5.16449764   2.72149425
   9.00309671   

[  6.58115181  13.66384507  11.36231004  19.09231707   2.03346189
   6.69458869   8.72227633   6.92174726   5.96807426   3.10435963
  10.70359782   9.76789376   3.86297985   0.37665392  -9.29685079] 158.024064953
[  6.59152623  13.71739287  11.41031696  19.17884098   2.03577924
   6.70852355   8.74352379   6.94130226   5.98503698   3.11012469
  10.74280983   9.812943     3.87076974   0.37538361  -9.30682531] 157.820359998
[  6.60167718  13.77051753  11.45797511  19.26476706   2.03806715
   6.72223619   8.76434863   6.96055899   6.00174694   3.11579988
  10.78163699   9.85765063   3.87843668   0.37411093  -9.31671657] 157.620327773
[  6.61161057  13.82322634  11.50529045  19.35010512   2.04032679
   6.73573457   8.78476546   6.9795263    6.0182099    3.12138863
  10.82008595   9.90202213   3.88598517   0.37283568  -9.32652584] 157.423849899
[  6.62133214  13.87552639  11.55226876  19.43486473   2.04255924
   6.74902603   8.80478826   6.99821264   6.03443145   3.12689405
  10.85816321   

[  6.8669706   15.53407009  13.05809459  22.16577601   2.1099854
   7.1281359    9.34358501   7.53380829   6.49398747   3.28546216
  12.02606299  11.34685045   4.10664092   0.32233665  -9.6414669 ] 151.94441905
[  6.87166809  15.57498044  13.09564115  22.23418253   2.11159075
   7.13661866   9.35501825   7.5458564    6.50407138   3.28905406
  12.0538715   11.38142252   4.11145319   0.32083659  -9.64889547] 151.831752893
[  6.8762823   15.61565834  13.13299391  22.30224951   2.11318517
   7.14501791   9.36631681   7.55778925   6.51404252   3.29261287
  12.08147327  11.41579461   4.11622067   0.31932635  -9.65627627] 151.720480789
[  6.88081512  15.65610674  13.17015527  22.369981     2.11476888
   7.15333523   9.37748375   7.56960896   6.52390284   3.29613924
  12.10887107  11.44996895   4.12094424   0.31780562  -9.66360988] 151.610577619
[  6.8852684   15.69632856  13.20712758  22.43738098   2.1163421
   7.16157215   9.38852205   7.58131763   6.53365425   3.29963376
  12.13606764  11.4

     fun: 149.57936055004308
     jac: array([  0.10854586,  -0.17425078,  -0.1618198 ,  -0.32657714,
         0.95816557,   0.4894419 ,   0.17578377,   0.15727232,
         0.01900598,   0.39131196,  -0.06387591,  -0.07476893,
         1.13804769, -15.62243738,   0.02442495])
 message: 'Finished iterations.'
  status: 'Finished iterations.'
 success: True
       x: array([  6.94922361e+00,   1.63077478e+01,   1.37714939e+01,
         2.34671449e+01,   2.15941808e+00,   7.29334780e+00,
         9.55326409e+00,   7.75820765e+00,   6.67665539e+00,
         3.35949198e+00,   1.25446074e+01,   1.20011628e+01,
         4.21806257e+00,  -2.24677501e-02,  -9.78109125e+00])

In [36]:
y, _ = m.predict_y(X_train)
print("Train:", r_squared(y_train, y.flatten()))
y, _ = m.predict_y(X_test)
print("Test:", r_squared(y_test, y.flatten()))

Train: 0.999999960497
Test: 0.91526999934
